# Generador de Python API

## Utilities

### Global imports

In [1]:
import django
import textwrap
import os
import re

### Common functions

In [2]:
def camel2snake(string):
    regex = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', string)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', regex).lower()


def get_internal_type(field):
    try:
        return field.get_internal_type()
    except Exception as _:
        return field.__class__.__name__    

    
def get_related_model(field):
    try:
        return field.related_model._meta.label
    except Exception as _:
        return None

## Generador de Serializer

In [3]:
from django_measurement.models import MeasurementField
from django_prices.models import MoneyField
from django_prices.models import TaxedMoneyField


def generateSerializer(model, destination_dir):
    file_name = camel2snake(model_name)
    
    fields = model._meta.get_fields()
    fields = dict(map(lambda f: (f.name, f), fields))
    fields = list(map(lambda f: fields[f], sorted(fields.keys())))

    reverse_fields = [f for f in fields if f.auto_created and not f.concrete]
    fields = [f for f in fields if f not in reverse_fields]
    money_fields = [f for f in fields if isinstance(f, MoneyField)]
    taxed_money_fields = [f for f in fields if isinstance(f, TaxedMoneyField)]
    measurement_fields = [f for f in fields if isinstance(f, MeasurementField)]
    custom_fields = []
        
    if money_fields:
        custom_fields.append('MoneyField')
        
    if taxed_money_fields:
        custom_fields.append('TaxedMoneyField')
        
    if measurement_fields:
        custom_fields.append('MeasurementField')
        
    # *** Serializers ***
    serializer = f"""    from django.apps import apps
    from rest_flex_fields import FlexFieldsModelSerializer"""
    
    if custom_fields:
        custom_fields = ", ".join(custom_fields)    
        serializer += f"""
    from ..fields import {custom_fields}
    """ 
    
    serializer += f"""

    __all__ = [
        '{model_name}Serializer',
    ]

    {model_name} = apps.get_model(*'{model._meta.label}'.split())


    class {model_name}Serializer(FlexFieldsModelSerializer):
        \"\"\"Serializer for :model:`{model._meta.label}`:

        `**Fields:**`"""

    for j, field in enumerate(fields, 1):
        field_name = f"`{field.name}`"
        field_type = get_internal_type(field)
        field_model_related = get_related_model(field)

        serializer += f"""
            {j:0>2d}. {field_name:30s}: `{field_type}`"""

        if field_model_related is not None:
            serializer += f""" [:model:`{field_model_related}`]"""

    serializer += """

        `**Reverse Fields:**`"""

    for j, field in enumerate(reverse_fields, 1):
        field_name = f"`{field.name}`"
        field_type = get_internal_type(field)
        field_model_related = get_related_model(field)

        serializer += f"""
            {j:0>2d}. {field_name:30s}: `{field_type}`"""

        if field_model_related is not None:
            serializer += f""" [:model:`{field_model_related}`]"""
    
    serializer += f"""
        \"\"\""""
    
    for field in money_fields: 
        serializer += f"""
        {field.name} = MoneyField()"""
    
    for field in taxed_money_fields: 
        serializer += f"""
        {field.name} = TaxedMoneyField()"""
    
    for field in measurement_fields: 
        serializer += f"""
        {field.name} = MeasurementField()"""
        
    serializer += f"""

        class Meta:
            model = {model_name}
            fields = [
                # Fields"""

    for _, field in enumerate(fields):
        serializer += f"""
                '{field.name}',"""

    serializer += """

                # Reverse Fields"""
    for _, field in enumerate(reverse_fields):
            serializer += f"""
                # '{field.name}',"""

    serializer += """
            ]
            read_only_fields = []

        # def create(self, validated_data):
        #     return super().create(validated_data)

        # def update(self, instance, validated_data):
        #     return super().update(instance, validated_data)
    """

    serializer_file = os.path.join(destination_dir, file_name + ".py")

    with open(serializer_file, 'w') as file:
        file.write(textwrap.dedent(serializer))
    file.close()

    return file_name, f'{model_name}Serializer'

## Generador de Viewset

In [4]:
def generateViewset(model, destination_dir):
    # model_name = model.__name__.replace("Bss", "")
    model_name = model.__name__
    
    file_name = camel2snake(model_name)
    path_name = camel2snake(model_name).replace('_','-')

    # *** Viewset ***
    fields = model._meta.get_fields()
    fields = dict(map(lambda f: (f.name, f), fields))
    fields = list(map(lambda f: fields[f], sorted(fields.keys())))

    reverse_fields = [f for f in fields if f.auto_created and not f.concrete]
    fields = [f for f in fields if f not in reverse_fields]
    lookup_field = getattr(model, 'lookup_field', 'id')
    
    viewset = f"""    from django.apps import apps
    from rest_framework.filters import SearchFilter
    from rest_framework.filters import OrderingFilter
    from django_filters.rest_framework import DjangoFilterBackend
    from rest_flex_fields import FlexFieldsModelViewSet
    from saleor.rest.serializers import {model_name}Serializer


    __all__ = [
        '{model_name}ViewSet',
    ]

    {model_name} = apps.get_model(*'{model._meta.label}'.split())

    
    class {model_name}ViewSet(FlexFieldsModelViewSet):
        \"\"\"ViewSet for :model:`{model._meta.label}`

        `** Actions **`:

        create:
        Create a new `{model._meta.label}` instance.

        retrieve:
        Return the given `{model._meta.label}`.

        update:
        Update the given `{model._meta.label}`..

        delete:
        Delete the given `{model._meta.label}`, and return an empty response 
        with HTTP 204 status code.

        list:
        Return a list of all the existing :model:`{model_name}`.
        \"\"\"
        
        lookup_field = '{lookup_field}'
        queryset = {model_name}.objects.all()
        serializer_class = {model_name}Serializer
        filter_backends = [DjangoFilterBackend, SearchFilter, OrderingFilter, ]
        filter_fields = [
            # Fields"""
    
    
    for _, field in enumerate(fields):
        field_type = get_internal_type(field)
        field_model_related = get_related_model(field)

        if field_model_related is None:
            viewset += f"""
            # '{field.name}',"""
        else:
            viewset += f"""
            # '{field.name}', # [{field_model_related}]"""

    viewset += """

            # Reverse Fields"""

    for _, field in enumerate(reverse_fields):
        viewset += f"""
            # '{field.name}',"""

    viewset += f"""
        ]
        search_fields = [
            # Fields"""

    for _, field in enumerate(fields):
        viewset += f"""
            # '{field.name}',"""

    viewset += """

            # Reverse Fields"""

    for _, field in enumerate(reverse_fields):
        viewset += f"""
            # '{field.name}',"""

    viewset += f"""
        ]
        ordering_fields = [
            # Fields"""

    for _, field in enumerate(fields):
        viewset += f"""
            # '{field.name}',"""

    viewset += """

            # Reverse Fields"""

    for _, field in enumerate(reverse_fields):
        viewset += f"""
            # '{field.name}',"""

    viewset += f"""
        ] 
        # '__all__'

        # def get_object(self):
        #     return super().get_object()

        # def create(self, request, *args, **kwargs):
        #     return super().create(request, *args, **kwargs)

        # def retrieve(self, request, *args, **kwargs):
        #     return super().retrieve(request, *args, **kwargs)

        # def update(self, request, *args, **kwargs):
        #    return super().update(request, *args, **kwargs)

        # def destroy(self, request, *args, **kwargs):
        #     return super().destroy(request, *args, **kwargs)

        # def list(self, request, *args, **kwargs):
        #     return super().list(request, *args, **kwargs)
    """

    viewset_file = os.path.join(destination_dir, file_name + ".py")

    with open(viewset_file, 'w') as file:
        file.write(textwrap.dedent(viewset))
    file.close()
    
    return file_name, path_name, f'{model_name}ViewSet'

## Generador global: All in one!

In [5]:
from django.db.models import Model

models = django.apps.apps.get_models(include_auto_created=False)
models = filter(lambda m: issubclass(m, Model), models)
models = filter(lambda m: not m._meta.abstract, models)
models = dict(map(lambda m: (m.__name__.replace("Bss", ""), m), models))
models = list(map(lambda n: (n, models[n]), sorted(models.keys())))

base_dir = 'autogenerated/rest'
serializers_dir = os.path.join(base_dir, 'serializers')
viewsets_dir = os.path.join(base_dir, 'views')

if not os.path.exists(base_dir):
    os.makedirs(base_dir)

if not os.path.exists(serializers_dir):
    os.mkdir(serializers_dir)
    
if not os.path.exists(viewsets_dir):
    os.mkdir(viewsets_dir)

model_exclusion_list = []
viewset_exclusion_list = []

serializers_init_file = os.path.join(serializers_dir, '__init__.py')
serializers_init_file = open(serializers_init_file, "w")

viewsets_init_file = os.path.join(viewsets_dir, '__init__.py')
viewsets_init_file = open(viewsets_init_file, "w")

for i, (model_name, model) in enumerate(models, 1):
    module = model._meta.app_label

    module_dir = os.path.join(serializers_dir, module)
    init_file = os.path.join(module_dir, '__init__.py')
    if not os.path.exists(module_dir):
        os.mkdir(module_dir)
        open(init_file, 'a').close()

    if model_name not in model_exclusion_list:
        file_name, serializer_name = generateSerializer(model, module_dir)
        init_file = open(init_file, 'a')
        init_file.write(f"from .{file_name} import {serializer_name}\n")
        init_file.close()
        serializers_init_file.write(f"from .{module} import {serializer_name}\n")

    module_dir = os.path.join(viewsets_dir, module)
    init_file = os.path.join(module_dir, '__init__.py')
    if not os.path.exists(module_dir):
        os.mkdir(module_dir)
        open(init_file, 'a').close()

    if model_name not in viewset_exclusion_list:
        file_name, path_name, viewset_name = generateViewset(model, module_dir)
        init_file = open(init_file, 'a')
        init_file.write(f"from .{file_name} import {viewset_name}\n")
        init_file.close()
        viewsets_init_file.write(f"from .{module} import {viewset_name}\n")

        # print(f"Suggested path to access viewset")
        print(f"* diver_urls.register(r\'^{path_name}\', {viewset_name})")

serializers_init_file.close()
viewsets_init_file.close()

* diver_urls.register(r'^address', AddressViewSet)
* diver_urls.register(r'^association', AssociationViewSet)
* diver_urls.register(r'^attribute', AttributeViewSet)
* diver_urls.register(r'^attribute-translation', AttributeTranslationViewSet)
* diver_urls.register(r'^attribute-value', AttributeValueViewSet)
* diver_urls.register(r'^attribute-value-translation', AttributeValueTranslationViewSet)
* diver_urls.register(r'^authorization-key', AuthorizationKeyViewSet)
* diver_urls.register(r'^bank-account', BankAccountViewSet)
* diver_urls.register(r'^banner', BannerViewSet)
* diver_urls.register(r'^benefit', BenefitViewSet)
* diver_urls.register(r'^benefit-translation', BenefitTranslationViewSet)
* diver_urls.register(r'^brand', BrandViewSet)
* diver_urls.register(r'^brand-translation', BrandTranslationViewSet)
* diver_urls.register(r'^category', CategoryViewSet)
* diver_urls.register(r'^category-translation', CategoryTranslationViewSet)
* diver_urls.register(r'^checkout', CheckoutViewSet)